## 모듈 설치

In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.25.2', '1.5.3')

## 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 준비 및 전처리

In [130]:
check_list = pd.read_csv('/content/drive/MyDrive/멋사/Team Space/data/raw_data/간편식_구입시_확인사항.csv', encoding='cp949') # 데이터 불러오기
check_list = check_list.drop('특성별(1)', axis=1) # 특성별(1) 열 삭제
check_list = check_list.rename(columns={'특성별(2)': 'year'}) # 열 이름 변경
check_list.columns = check_list.columns.map(lambda x: x.split('.')[0]) # 연도 '.' 뒤에 삭제
drop_index_list = [8, 9] + list(range(26, 34)) # 필요없는 행 삭제(읍, 동, 학력, 직업)
check_list = check_list.drop(drop_index_list)
check_list = check_list.T.reset_index() # 행렬 변환
check_list.head(10)

string_to_remove = '-' # 결측값 '-'으로 표시되어 있음
check_list = check_list.applymap(lambda x: x.replace(string_to_remove, '0')) # '-'값 찾아서 '0'으로 변환 -> 뒤에 계산해야 함.
check_list.columns = check_list.iloc[0]
check_list = check_list.drop(check_list.index[0])
check_list = check_list.rename(columns={'특성별(2)': '항목'})
# check_list

for col in check_list.columns[2:]: # 2번쨰 열('소계')부터
    check_list[col] = pd.to_numeric(check_list[col]) # float 형식으로 변환

# 열 이름 통합

# 비수도권 통합
check_list['비수도권'] = (check_list['비수도권'] + check_list['호남권'] + check_list['대경권'] + check_list['동남권']+ check_list['강원권']) / 3 # 400~500, 400~600, 500~600 열 값 더하고 나누기 3 -> 다시 400~600으로 지정하여 저장
check_list.drop(columns=['호남권'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['대경권'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['동남권'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['강원권'], inplace=True) # 기존 열 삭제

# 가구원수별 통합
check_list['2~3인'] = (check_list['2인'] + check_list['2~3인'] + check_list['3인']) / 3 # 2인, 2~3인, 3인 열 값 더하고 나누기 3 -> 다시 2~3인으로 지정하여 저장
check_list.drop(columns=['2인'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['3인'], inplace=True) # 기존 열 삭제

check_list['4인 이상'] = (check_list['4인'] + check_list['4인 이상'] + check_list['5인 이상']) / 3 # 4인, 4인 이상, 5인 이상의 열 값 더하고 나누기 3 -> 다시 4인 이상으로 지정하여 저장
check_list.drop(columns=['4인'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['5인 이상'], inplace=True) # 기존 열 삭제

# 연령대 통합
check_list['20~30대'] = (check_list['20대'] + check_list['20~30대'] + check_list['30대']) / 3
check_list.drop(columns=['20대'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['30대'], inplace=True) # 기존 열 삭제

check_list['40~50대'] = (check_list['40대'] + check_list['40~50대'] + check_list['50대']) / 3
check_list.drop(columns=['40대'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['50대'], inplace=True) # 기존 열 삭제

# 소득 통합
check_list['200~400만원 미만'] = (check_list['200~300만원 미만'] + check_list['200~400만원 미만'] + check_list['300~400만원 미만']) / 3 # 200~300, 300~400 열 값 더하고 나누기 2 -> 200~400으로 지정하여 저장
check_list.drop(columns=['200~300만원 미만'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['300~400만원 미만'], inplace=True) # 기존 열 삭제

check_list['400~600만원 미만'] = (check_list['400~500만원 미만'] + check_list['400~600만원 미만'] + check_list['500~600만원 미만']) / 3 # 400~500, 400~600, 500~600 열 값 더하고 나누기 3 -> 다시 400~600으로 지정하여 저장
check_list.drop(columns=['400~500만원 미만'], inplace=True) # 기존 열 삭제
check_list.drop(columns=['500~600만원 미만'], inplace=True) # 기존 열 삭제

check_list = check_list.apply(lambda x: x.round(2) if x.dtype=='float' else x) # 둘째 자리에서 반올림

# 2021년 합산할 행들 선택
rows_to_mean = [41, 44, 45, 46]

# 새로운 행 생성 (첫번째, 두번째 열의 이름과 값 지정, 3번째 열부터 값 평균 구하기)
new_row_values_1 = check_list.iloc[rows_to_mean, 2:].mean()

# 첫번째와 두번째 열의 값 지정
new_row_values_1['year'] = '2021'
new_row_values_1['항목'] = '기타'

# 새로운 행 삽입 (기존 행 삭제 후 새로운 행을 삽입)
check_list = pd.concat([check_list.iloc[:46], pd.DataFrame([new_row_values_1], index=[46]), check_list.iloc[46:]]).reset_index(drop=True)
drop_row_list = [41, 44, 45, 47]
check_list = check_list.drop(drop_row_list)
check_list.reset_index(drop=True, inplace=True)

# 2022년 합산할 행들 선택
rows_to_mean = [52, 55, 56]

# # 새로운 행 생성 (첫번째, 두번째 열의 이름과 값 지정, 3번째 열부터 값 평균 구하기)
new_row_values = check_list.iloc[rows_to_mean, 2:].mean()

# 첫번째와 두번째 열의 값 지정
new_row_values['year'] = '2022'
new_row_values['항목'] = '기타'
new_row_values

# 새로운 행 삽입 (기존 행 삭제 후 새로운 행을 삽입)
check_list = pd.concat([check_list.iloc[:57], pd.DataFrame([new_row_values], index=[57]), check_list.iloc[57:]]).reset_index(drop=True)
drop_row_list = [52, 55, 56]
check_list = check_list.drop(drop_row_list)
check_list.reset_index(drop=True, inplace=True)

check_list = check_list.apply(lambda x: x.round(2) if x.dtype=='float' else x)

# 2018-2020 : 기능성/식품의 효능 / 2021-2022 : 기능성 표시/식품의 효능 -> 기능성/식품의 효능 이름으로 통일

# 특정 인덱스들 지정(2021년, 2022년 행)
rename_index_list = [42, 53]

# 동일한 이름으로 수정할 값 설정
new_name_value = '기능성/식품의 효능'

# 각 인덱스에 대해 해당 인덱스의 첫 번째 값을 수정할 값으로 변경하기
for indexs in rename_index_list:
    first_value = check_list.iloc[indexs]['항목']
    check_list.at[indexs, '항목'] = new_name_value

check_list
check_list.to_csv('간편식_구입_확인사항.csv', encoding='cp949', index=False) # csv 파일 저장. index 삭제